## Le mie finestre e le mie porte

Come sempre ho iniziato il progetto documentandomi su come dovrebbe essere fatta una finestra e, di conseguenza, una porta finestra. Ecco le mie immagini di riferimento:

![finestra esempio 1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/products-double-hung-window-2x.jpg?raw=true)
(la finestra sopra è stata ricreata, vedere la sezione risultati)
![finestra esempio 2](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/Custom-made-Aluminum-bay-window-for-sale.jpg?raw=true)


In [2]:
from pyplasm import *

Evaluating fenvs.py..
...fenvs.py imported in 0.006159 seconds


In [8]:
""" intRGBColor

Metodo preso dal web per ottenere colori da RGB
@param values: lista di tre interi RGB
@return: Colore
"""
def intRGBColor(values):
        return Color4f([ values[0]/255.0,
                         values[1]/255.0,
                         values[2]/255.0,
                         0.5])
GOLD=intRGBColor([238,232,170])
LIGHT_BLUE = intRGBColor([192,242,247])
LIGHT_BROWN = intRGBColor([193,157,98])

## l'algorimo per creare le finestre

Il metodo principale che si occupa di generare la finstra di riferimento è design_4_Windows; 
Il problema più grande è come passare i dati alla funzione, infatti per generare il reticolo di legno e vetro della finestra si devono passare quattro array: 
* X: le dimensioni su X di ogni pezzo di legno o di vetro
* Y: le dimensioni su Y di ogni pezzo di legno o di vetro
* Z: un array con un solo elemento, lo spessore della finestra
* mirror_not_mirorr: è un array che descrive se nella iesima-cella in posizione X,Y, c'è vetro (0) o c'è legno (1)

Il caso base della generazione del legno è presto risolto con un semplice check se in posizione i ci fosse 1 oppure 0.
Per quanto riguarda il caso del vetro e volendo riusare il codice, ho usato un po' di ricorsione: nello specifico ho passato il compito di inverire l'array  mirror_not_mirror alla funzione inverte_0_1 che dopo aver invertito tutto richiama la funzione design_4_Windows mettendo il tag booleano "ricorsione" a True, cosi da non entrare in un loop infinito. A questo punto ci si ritrova ad avere 2 hpc uno rappresentante la struttura in legno, l'altro quella in vetro (vedere l'immagine)

![risultati1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/fram3.png?raw=true)

Unite le due immagini non ci resta che triplicare l'hpc risultante e creare un hpc in cui le tre finestre siano ognuna a 45° dalla succesiva. Per rendere piu credibile la finestra, se c'è spazio (e lo si vede dall'array mirror_not_mirror) il metodo ausiliario create_openable_window crea una ulteriore finestra con maniglia e la pone sopra le finestre gia create, dopo aver tolto uno dei vetri. 
In Tutto questo è stata inserita anche la possibilità di usare i parametri dx,dy per scalare l'a finestra creata,l'accortezza è che vengono allargate soltanto le parti della finestra che non sono di congiunzione.

## Risultati 

![risultati1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/fram1.png?raw=true)

![risultati1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/fram4.png?raw=true)

![risultati1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/fram5.png?raw=true)

![risultati1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/fram8.png?raw=true)
(finestra scalata in altezza: dimensioni adeguate a quelle di una finestra di una cattedrale!)

![risultati1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/fram9.png?raw=true)

In [9]:
""" create_openable_window

Metodo ausiliare che genera una finestra e una maniglia per il metodo design_4_Windows

@param values: gli stessi di design_4_Windows, la maniglia d'aggiungere
@return: hpc di una finestra con maniglia
"""
def create_openable_window(X,Y,Z,dx,dy,handle,mirror_or_not_mirror):
    sum=0
    toReturn=[]
    ehigth=0
    handle=COLOR(GOLD)(STRUCT([T(3)(Z[0]),R([1,2])(math.pi/2),handle]))
    for i in range(len(mirror_or_not_mirror)):
        for j in range(len(mirror_or_not_mirror[i])):
            if(mirror_or_not_mirror[i][j]==1):
                sum=sum+1
        if(i%2==0 and sum==(len(mirror_or_not_mirror[i])+2)):
            
            xscale=[]
            for scale in range(len(X)):
                if(scale%2==0):
                    xscale.append(X[scale])
                else:
                    xscale.append(X[scale]+dx)
            lenX=0
            for h in xscale:
                lenX= lenX +h
            if(lenX>0):
                toAdd=CUBOID([lenX,Y[i],Z[0]])
                toReturn.append(toAdd)
                toReturn.append(T(2)(Y[i]))
                toAdd2=CUBOID([Z[0],Y[i+1]+dy,Z[0]])
                glass=COLOR(LIGHT_BLUE)(STRUCT([T(3)(Z[0]/2.0),CUBOID([(lenX-1*Z[0]),Y[i+1]+dy,Z[0]/10.0])]))
                toAdd2=STRUCT([toAdd2,T(1)(Z[0]),glass,T(1)(lenX-2*Z[0]),toAdd2,T(1)(-(lenX-1*Z[0]))])
                toReturn.append(toAdd2)
                toReturn.append(T(2)(Y[i+1]+dy))
                toReturn.append(toAdd)
                toReturn.append(T(2)(Y[i]))
                toReturn=STRUCT(toReturn)
                toReturn=STRUCT([toReturn,T([1,2])([(SIZE(1)(toReturn))/2.0,(SIZE(2)(toReturn)+ehigth)-Z[0]]),handle])
                return toReturn
                
        elif(i%2==0 and i+1!=len(Y)):
            sum=0
            ehigth=ehigth+Y[i]+Y[i+1]+dy
            toReturn.append(T(2)(Y[i]))
            toReturn.append(T(2)(Y[i+1]+dy))
    return 0  

In [10]:
"""
il seguente metodo genera un gruppo di tre finestre ognuna a 45° rispetto alla successiva;
se c'è spazio il primo vetro viene tolto e con create_openable_window si aggiunge una finestra
alzabile e abbasabile per chiudere la presa d'aria
@param X: array su X di dimesioni del legno
@param Y: array su Y di dimesioni del legno
@param Z: array su Z di spessori del legno
@param mirror_or_not_mirror: matrice che dice dove c'è legno 1 e dove c'è vetro 0
@return: hpc della finestra
"""
"""ogni volta controlla per ogni elemento se deve inserirlo e di quanto deve essere largo su X e su Y, poi lo estrude"""
"""si cicla su X che puo avere valori positivi o negativi a seconda se c'è legno o meno, quindi ogni volta si usa il suo valore assoluto """   
""" alla prima chiamata recursive è falso, qiundi si esegue questo passo una sola voltaqui viene richiamata la funzione di inversione che richiama ricorsivamente questa funzione con recursion true al finedi generare il vetro"""    
"""vengono triplicate le strutture create"""            
"""nel passo ricorsivo si ritorna la singola finestra"""   
"""altrimenti quando recursive è false si ritorna l'hpc delle tre finestre"""

def design_4_Windows(X,Y,Z,mirror_or_not_mirror,recursive=False):        
    def scale_and_rotate(dx=0,dy=0,dz=0):
        final_struct=[]
        sum=0
        window_and_handle=[]
        handle=STRUCT([CUBOID([Z[0]*2,Z[0],Z[0]]),T([1,3])([Z[0],Z[0]]),CUBOID([Z[0],Z[0]*5,Z[0]]),
                       T([1,2,3])([-Z[0],Z[0]*4,-Z[0]]),CUBOID([Z[0]*2,Z[0],Z[0]])])


        for i in range(len(mirror_or_not_mirror)):
            if(i==1 and recursive==False):
                window_and_handle= create_openable_window(X,Y,Z,dx,dy,handle,mirror_or_not_mirror)
            for j in range(len(mirror_or_not_mirror[i])):
                scaleY=0 
                if (mirror_or_not_mirror[i][j]==0):
                    X[j]= (-1.0)*abs(X[j])
                else:
                    sum=sum+1
                    X[j]=abs(X[j])              
            if(recursive and sum==len(Y) and i==1):
                scaleY=dy
            
            elif(sum!=0):
                xscale=[]
                for scale in range(len(X)):
                    dx=abs(dx)
                    if(X[scale]<0):
                        dx=-1.0*dx
                    if(scale%2==0):
                        xscale.append(X[scale])
                    else:
                        xscale.append(X[scale]+dx)      
                if(i%2!=0):
                    scaleY=dy
                toAdd= PROD([QUOTE(xscale),QUOTE([Y[i]+scaleY])])
                final_struct.append(toAdd)
            final_struct.append(T(2)(Y[i]+scaleY))
            sum=0
        
        final_struct=STRUCT(final_struct)
        final_struct=(PROD([final_struct,QUOTE(Z)]))
        if(recursive==False):
            mirror=COLOR(LIGHT_BLUE)(inverte_0_1(X,Y,Z,(dx,dy,dz),mirror_or_not_mirror))
            final_struct=STRUCT([final_struct,T(3)(Z[0]/2.0),mirror])
            if((window_and_handle)!=0):
                final_struct=STRUCT([final_struct,T([2,3])([-Y[1]/3.0-dy,Z[0]]),window_and_handle])
    
        window1=final_struct
        window2=final_struct
        window3=final_struct 
        separator= CUBOID([Z[0]*3,SIZE(2)(window2),Z[0]]);
        
        window2=STRUCT([T([1,3])([Z[0],0]),window2])
        window2=(STRUCT([R([1,3])(math.pi*1/4),(window2)]))
        window3=STRUCT([R([1,3])(-math.pi*1/4),(window1)])
            
        value=math.sqrt(Z[0]*Z[0]*2)
        window3=STRUCT([T([1,3])([0,SIZE(3)(window3)-value]),window3])
        
        window1=STRUCT([separator,T([1,3])([Z[0]*3,0]),window1,T(1)(SIZE(1)(window1)),separator])
        window1=STRUCT([window3,T([1])([SIZE(1)(window3)-Z[0],0]),window1])
        window1=STRUCT([T(3)(Z[0]),window1])
        window1=STRUCT([window1,T([1,3])([SIZE(1)(window1)-value/4.0,+value/4.0]),window2])

        if(recursive):
            return final_struct 
        else:
            return window1

    return scale_and_rotate


"""inverte la matrice e chima design_4_Windows con il booleano recursion a true""" 
def inverte_0_1(X,Y,Z,(dx,dy,dz),listOfList):
    for i in range(len(listOfList)):
        for j in range(len(listOfList[i])):
            if(listOfList[i][j]==0):
                listOfList[i][j]=1
            else:
                listOfList[i][j]=0
    if(i==0):
        return []

    valueZ=Z[0]
    return design_4_Windows(X,Y,[valueZ/10],listOfList,True)(dx,dy,dz)


## Generatore di Porte

Per quanto riguarda la generazione di porte ho cercato il piu possibile di pulire il codice che generava le finestra, lasciandovi l'essenziale per generarela sola struttura in legno vetro della porta-finestra. Dopo di che la inserisco all'interno dell'intercapedine della porta e ci attacco la maniglia.

In [18]:
"""
il seguente metodo genera una porta e la inserisce in una struttura che la contenga.
Il metodo è una versione semplificata di quello della finestra
@param X: array su X di dimesioni del legno
@param Y: array su Y di dimesioni del legno
@param Z: array su Z di spessori del legno
@param mirror_or_not_mirror: matrice che dice dove c'è legno 1 e dove c'è vetro 0
@return: hpc della porta
"""

"""nel passo non ricorsivo si adorna la porta con la maniglia e gli si chiede di generare i vetri sempre con la funzione inverte_0_1 che rientra in questa funzione ma con il boolean recursive a True"""

def design_4_door(X,Y,Z,mirror_or_not_mirror,recursive=False):     
    def scale_door(dx,dy,dz):
        final_struct=[]
        sum=len(Y)
        handle=STRUCT([CUBOID([Z[0]*2,Z[0],Z[0]]),T([1,3])([Z[0],Z[0]]),CUBOID([Z[0],Z[0]*5,Z[0]]),
                       T([1,2,3])([-Z[0],Z[0]*4,-Z[0]]),CUBOID([Z[0]*2,Z[0],Z[0]])])
        
        for i in range(1,len(mirror_or_not_mirror)):
            for j in range(len(mirror_or_not_mirror[i])):
                scaleY=0 
                if (mirror_or_not_mirror[i][j]==0):
                    X[j]= (-1.0)*abs(X[j])
                else:
                    sum=sum+1
                    X[j]=abs(X[j])        
            
            if(sum!=0):
                xscale=[]
                for scale in range(len(X)):
                    dx=abs(dx)
                    if(X[scale]<0):
                        dx=-1.0*dx
                    if(scale%2==0):
                        xscale.append(X[scale])
                    else:
                        xscale.append(X[scale]+dx)      
                if(i%2!=0):
                    scaleY=dy
                toAdd= PROD([QUOTE(xscale),QUOTE([Y[i]+scaleY])])
                final_struct.append(toAdd)
            final_struct.append(T(2)(Y[i]+scaleY))
            sum=0
        
        final_struct=STRUCT(final_struct)
        final_struct=(PROD([final_struct,QUOTE(Z)]))

        if(recursive==False):
            mirror=COLOR(LIGHT_BLUE)(inverte_0_1(X,Y,Z,(dx,dy,dz),mirror_or_not_mirror))
            handle=COLOR(GOLD)(handle)
            final_struct=COLOR(LIGHT_BROWN)(final_struct)
            sizeOnY=SIZE(2)(final_struct)
            moveOnX=X[0]/2.0+Z[0]
            final_struct=STRUCT([final_struct,T([2,3])([-Y[0],Z[0]/2.0]),mirror,T([1,2])([-moveOnX,sizeOnY/2.0+Y[0]]),handle])
            doorContainer=(SIZE(1)(final_struct),SIZE(2)(final_struct))
            doorContainer=OFFSET([Z[0],Z[0],Z[0]*3])(SKEL_1(CUBOID([doorContainer[0]+Z[0],doorContainer[1]+Z[0],0])))
            final_struct=STRUCT([doorContainer,T([1,2,3])([Z[0],Z[0],Z[0]]),final_struct])

        return final_struct


    return scale_door

## Risultati Porta

![risultati1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/fram6.png?raw=true)

![risultati1](https://github.com/adixia/ggpl-LAB/blob/master/2016-11-25/fram7.png?raw=true)


In [19]:
X=[1,5,1,5,1,5,1,5,1]
Y=[1,10,1,10,1,10,1]
Z=[0.5]
mirror_or_not_mirror=[
    [1,1,1,1,1,1,1,1,1],
    [1,0,0,0,0,0,0,0,1],
    [1,1,1,1,1,1,1,1,1],
    [1,0,0,0,0,0,0,0,1],
    [1,1,1,1,1,1,1,1,1],
    [1,0,1,0,1,0,1,0,1],
    [1,1,1,1,1,1,1,1,1]]

X1=[5,5,1,5,1,5,1]
Y1=[1,5,1,5,1,5,1,5,1,5,1]
Z1=[0.5]
mirror_or_not_mirror1=[
    [1,1,1,1,1,1,1],
    [1,0,1,0,1,0,1],
    [1,0,1,0,1,1,1],
    [1,0,1,0,1,0,1],
    [1,0,1,0,1,1,1],
    [1,0,1,0,1,0,1],
    [1,0,1,0,1,1,1],
    [1,0,1,0,1,0,1],
    [1,0,1,0,1,1,1],
    [1,0,1,0,1,0,1],
    [1,1,1,1,1,1,1]]
X2=[1.5,5,1,5,1]
Y2=[10,5,1,5,1,5,1,5,1]
Z2=[0.5]
mirror_or_not_mirror2=[
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1]]
VIEW(design_4_Windows(X,Y,Z,mirror_or_not_mirror)(0,0,1))
VIEW(design_4_door(X1,Y1,Z1,mirror_or_not_mirror1)(0.1,1,0.1))
VIEW(design_4_door(X2,Y2,Z2,mirror_or_not_mirror2)(0,0,0.1))
mirror_or_not_mirror2=[
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1],
    [1,0,1,0,1],
    [1,1,1,1,1]]
VIEW(design_4_Windows(X2,Y2,Z2,mirror_or_not_mirror2)(1,1,1))

<pyplasm.xgepy.Hpc; proxy of <Swig Object of type 'std::shared_ptr< Hpc > *' at 0x1045df6f0> >